In [5]:
import pandas as pd

orders_df = pd.read_csv('../data/orders.csv')


print(orders_df.shape)
orders_df.head()

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
print("the largest days_since_prior_order:", orders_df['days_since_prior_order'].max())
print("Number customers has the first order:", orders_df['days_since_prior_order'].isnull().sum())
print("Number of unique user_uid:", orders_df['user_id'].nunique())

the largest days_since_prior_order: 30.0
Number customers has the first order: 206209
Number of unique user_uid: 206209


In [ ]:
import numpy as np
from datetime import datetime, timedelta

# Set random seed for reproducibility
np.random.seed(42)

# Define date range
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 11, 1)
date_range_days = (end_date - start_date).days

# Step 1: Generate random dates for each user's first order
first_orders = orders_df[orders_df['days_since_prior_order'].isnull()].copy()
num_users = len(first_orders)

# Generate random dates
random_days = np.random.randint(0, date_range_days, size=num_users)
random_dates = [start_date + timedelta(days=int(days)) for days in random_days]

# Create a mapping of user_id to first_order_date
user_first_date_map = dict(zip(first_orders['user_id'], random_dates))

# Step 2: Add first_order_date to all rows
orders_df['first_order_date'] = orders_df['user_id'].map(user_first_date_map)

# Step 3: Calculate days_since_first_order (cumulative sum per user)
orders_df = orders_df.sort_values(['user_id', 'order_number'])
orders_df['days_since_first_order'] = orders_df.groupby('user_id')['days_since_prior_order'].transform(lambda x: x.fillna(0).cumsum())

# Step 4: Calculate order_date for all orders
orders_df['order_date'] = orders_df['first_order_date'] + orders_df['days_since_first_order'].astype(int).apply(lambda x: timedelta(days=x))


In [ ]:
# orders_df[orders_df['days_since_first_order'].astype(int) > 100]
orders_df['days_since_first_order'].max()

365.0

In [ ]:
orders_df['']

In [ ]:
pd.set_option('display.max_rows', 1000)

orders_df['order_date'].value_counts().sort_index()

In [ ]:
print(f"Assigned random first order dates to {num_users} users")
orders_df[['order_id', 'user_id', 'order_number', 'days_since_prior_order', 'days_since_first_order', 'first_order_date', 'order_date']].head(20)

Assigned random first order dates to 206209 users


,order_id,user_id,order_number,days_since_prior_order,days_since_first_order,first_order_date,order_date
0,2539329,1,1,NaN,0.0,2025-04-13,2025-04-13
1,2398795,1,2,15.0,15.0,2025-04-13,2025-04-28
2,473747,1,3,21.0,36.0,2025-04-13,2025-05-19
3,2254736,1,4,29.0,65.0,2025-04-13,2025-06-17
4,431534,1,5,28.0,93.0,2025-04-13,2025-07-15
5,3367565,1,6,19.0,112.0,2025-04-13,2025-08-03
6,550135,1,7,20.0,132.0,2025-04-13,2025-08-23
7,3108588,1,8,14.0,146.0,2025-04-13,2025-09-06
8,2295261,1,9,0.0,146.0,2025-04-13,2025-09-06
9,2550362,1,10,30.0,176.0,2025-04-13,2025-10-06


In [ ]:

# Show sample data for a few users
print("Sample data for user_id 1:")
orders_df[orders_df['user_id'] == 1][['order_id', 'user_id', 'order_number', 'days_since_prior_order', 'order_date']].head(10)

Sample data for user_id 1:


,order_id,user_id,order_number,days_since_prior_order,order_date
0,2539329,1,1,NaN,2025-04-13
1,2398795,1,2,15.0,2025-04-28
2,473747,1,3,21.0,2025-05-19
3,2254736,1,4,29.0,2025-06-17
4,431534,1,5,28.0,2025-07-15
5,3367565,1,6,19.0,2025-08-03
6,550135,1,7,20.0,2025-08-23
7,3108588,1,8,14.0,2025-09-06
8,2295261,1,9,0.0,2025-09-06
9,2550362,1,10,30.0,2025-10-06


In [ ]:
print("\nSample data for user_id 2:")
orders_df[orders_df['user_id'] == 2][['order_id', 'user_id', 'order_number', 'days_since_prior_order', 'order_date']].head(10)


Sample data for user_id 2:


,order_id,user_id,order_number,days_since_prior_order,order_date
11,2168274,2,1,NaN,2025-09-28
12,1501582,2,2,10.0,2025-10-08
13,1901567,2,3,3.0,2025-10-11
14,738281,2,4,8.0,2025-10-19
15,1673511,2,5,8.0,2025-10-27
16,1199898,2,6,13.0,2025-11-09
17,3194192,2,7,14.0,2025-11-23
18,788338,2,8,27.0,2025-12-20
19,1718559,2,9,8.0,2025-12-28
20,1447487,2,10,6.0,2026-01-03


In [ ]:
orders_df[orders_df['order_date'] == '2026-01-04']['order_hour_of_day'].value_counts().sort_index()

order_hour_of_day
0      48
1      14
2      17
3       8
4       7
5      18
6      87
7     184
8     416
9     515
10    627
11    602
12    572
13    623
14    582
15    611
16    555
17    514
18    357
19    322
20    220
21    169
22    114
23     80
Name: count, dtype: int64

In [ ]:
orders_df.to_csv('../data/orders_with_dates.csv', index=False)

In [ ]:
orders_df.shape

NameError: name 'orders_df' is not defined